In [1]:
from network_models.soundsream_models_and_utils.clip_like.mapping_down.ss_direct_downmapping_with_conv import SS_Direct_Downmapping_Conv_Model
import importlib

import numpy as np

from network_models.soundsream_models_and_utils.clip_like.mapping_down.ss_direct_downmapping import SS_Direct_Downmapping_Model
from network_models.soundsream_models_and_utils.clip_like.encoder.ss_encoder_downmapping import EncoderDownmapping
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full
import torch
import gc

device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 8
models_dir = "/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/downmapping/0_0216/"
epochs =200
save_every = 40
start_lr = 6e-6
gc.collect()

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncodings_version0_12_1.pkl", device=device)

#model = SS_Direct_Downmapping_Model(dropout=0.2, output=1024, start_dim=512 * 175).to(device)
#model = SS_Direct_Downmapping_Conv_Model(dropout=0.2, output=1024, x_size=512, y_size=175).to(device)
model = EncoderDownmapping(embed_dim=512, n_heads=4, ff_dim=2, n_layers=1, dropout=0.2, output=1024, max_seq_len=175).to(device)

2023-02-18 12:57:10.632614: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 12:57:11.114950: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-18 12:57:11.115000: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-18 12:57:11.115004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [15]:
import sys
from pathlib import Path

module_path = str(Path.cwd().parents[0] / "network_models/soundstream_models_and_utils/encoder")
if module_path not in sys.path:
    sys.path.append(module_path)
import network_models.soundsream_models_and_utils.clip_like.mapping_down.ss_driect_dm_trainer as sset
importlib.reload(sset)

epochs =500
enc_trainer = sset.SSDirectDMTrainer(is_encoder=True, batch_size=batch_size, num_epochs=epochs, model_path=models_dir, save_model_every=save_every, lr=start_lr, dataset=data_set, device=device, model=model)
gc.collect()
enc_trainer.train() # 200 +500 + 500 +500+500

Epoch 1
-------------------------------
loss: 1.165422  [    0/ 6092]
loss: 1.165422  [ 1400/ 6092]
loss: 1.165422  [ 2800/ 6092]
loss: 1.165429  [ 4200/ 6092]
loss: 1.165593  [ 5600/ 6092]
552.0717619657516
Epoch 2
-------------------------------
loss: 1.308279  [    0/ 6092]
loss: 1.165422  [ 1400/ 6092]
loss: 1.235397  [ 2800/ 6092]
loss: 1.181350  [ 4200/ 6092]
loss: 1.165422  [ 5600/ 6092]
552.816523194313
Epoch 3
-------------------------------
loss: 1.165423  [    0/ 6092]
loss: 1.165434  [ 1400/ 6092]
loss: 1.191912  [ 2800/ 6092]
loss: 1.178546  [ 4200/ 6092]
loss: 1.165581  [ 5600/ 6092]
552.5810016393661
Epoch 4
-------------------------------
loss: 1.165459  [    0/ 6092]
loss: 1.165426  [ 1400/ 6092]
loss: 1.165422  [ 2800/ 6092]
loss: 1.184098  [ 4200/ 6092]
loss: 1.166292  [ 5600/ 6092]
551.4457051753998
Epoch 5
-------------------------------
loss: 1.165428  [    0/ 6092]
loss: 1.217754  [ 1400/ 6092]
loss: 1.165446  [ 2800/ 6092]
loss: 1.167743  [ 4200/ 6092]
loss: 1.1

KeyboardInterrupt: 

In [2]:
import sys
from pathlib import Path

module_path = str(Path.cwd().parents[0] / "network_models/soundstream_models_and_utils")
if module_path not in sys.path:
    sys.path.append(module_path)

sys.path.insert(0, "/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject")
import network_models.soundsream_models_and_utils.ss_encoded_dataset as ssed

dataset = ssed.ss_encoded_dataset_full(
    directory_cafe="/home/ckwdani/Music/emotionDatasets/converted_mono/cafe",
    directory_tess="/home/ckwdani/Music/emotionDatasets/converted_mono/tess",
    directory_ravdess="/home/ckwdani/Music/emotionDatasets/converted_mono/RAVDESS Audio_Speech_Actors_01-24",
    directory_mesd="/home/ckwdani/Music/emotionDatasets/converted_mono/mesd",
    device="cuda",
    #sound_stream_path="../notebooks/content/soundstream/vers0.7.4/01_Soundstream_7_x_new_libri_full/5_3250/soundstream.3250.pt")
    #sound_stream_path="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/soundstream/vers0.7.4/01_Soundstream_7_x_new_libri_full/currentSelection/soundstream.72500.pt")
    sound_stream_path="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/soundstream/verision0.12.1/10_10000_1e-4_bs6_gae8_dml320-32/soundstream.8000.pt",
    clip_path="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/downmapping/0_0216/encoder_360.pth" # trained for 1500 epochs
)
dataset.saveEncoding("../notebooks/content/data/allEncodings_clip_like.pkl")

KeyboardInterrupt: 

In [33]:
# from network_models.soundsream_models_and_utils.ss___util_class_batches_sampler import ClassBatchesSampler
# np1 = data_set.encoded_dataset.encodedData[data_set.encoded_dataset.labelcolumn].to_numpy()
# from torch.utils.data import DataLoader
# #
# # l = [np1[i][0] for i in range(len(np1))]
# #
# #
# dl = DataLoader(data_set, batch_sampler=ClassBatchesSampler(np1, num_class_samples=2, shuffle=False))
# #
# #
# for batch, (X, z) in enumerate(dl):
#
#     print(torch.tensor_split(torch.argmax(z, dim=1), 2))
#     z1, z2 = torch.tensor_split(z, 2)
#     pred = model(X.cuda())
#     pred1, pred2 = torch.tensor_split(pred, 2)
#     dp = torch.matmul(pred1, pred2.t())
#     #dp = pred1 @ pred2.t()
#
#
#     print(torch.dot(pred1[0],pred2[0]))
#     # print(torch.dot(pred[1],pred2[0]))
#     # print(torch.dot(pred[2],pred2[0]))
#     # print(torch.dot(pred[3],pred2[0]))
#     # print(torch.dot(pred[4],pred2[0]))
#     # print(torch.dot(pred[5],pred2[0]))
#     # print(torch.dot(pred[7],pred[0]))
#     #dp = torch.matmul(z1, z2.T) / (z1.norm(dim=-1, keepdim=True) * z2.norm(dim=-1, keepdim=True))
#     #print(dp)
#     print(dp)
#     print(torch.sum(dp[0, :]))
#
#     #print(X[0][0][0][170])
#
#     if batch == 0:
#         break
#
# (len(dl)/(7*2))-2 == 462

(tensor([0, 1, 2, 3, 4, 5, 6]), tensor([0, 1, 2, 3, 4, 5, 6]))
tensor(0.0257, device='cuda:0', grad_fn=<DotBackward0>)
tensor([[ 0.0257,  0.0682,  0.1137,  0.1003,  0.0861,  0.0434,  0.1270],
        [ 0.0364,  0.1691,  0.0499,  0.0520,  0.0445,  0.0215,  0.0879],
        [ 0.0471,  0.2021,  0.1437,  0.1197,  0.0319,  0.0232,  0.0524],
        [-0.0349,  0.1412, -0.0580,  0.1619,  0.0691,  0.0561,  0.1994],
        [ 0.1065,  0.1545,  0.1924,  0.1211,  0.1037,  0.0426,  0.1208],
        [ 0.1051,  0.1988,  0.1682,  0.2198,  0.1852,  0.0045,  0.1342],
        [ 0.0185,  0.0735,  0.0420,  0.0426,  0.0786,  0.0026,  0.5665]],
       device='cuda:0', grad_fn=<MmBackward0>)
tensor(0.5645, device='cuda:0', grad_fn=<SumBackward0>)


True